In [1]:
import time
from pprint import pprint
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
def get_driver():
    options = Options()
    options.headless = True

    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    browser = webdriver.Firefox(options=options, firefox_profile=firefox_profile)

    return browser

In [3]:
def parsed_url(page = 1, low_range=250, up_range=5000, build_link = None):
    base_url = 'https://pcpartpicker.com'
    if build_link == None:
        fragment = f'/builds/#B=1&page={page}&X={low_range}00,{up_range}00'
    else: 
        fragment = f'{build_link}'

    return f'{base_url}{fragment}'

In [89]:
def build_scraper(url):
    builds_dict = {}
    build_comps = ['Name','CPU', 'CPU Cooler', 'Motherboard', 'Memory', 'Storage', 'Video Card', 'Case', 'Power Supply', 'Build Price']
    rq = requests.get(url)
    if rq.status_code != requests.codes.ok:
        return builds_dict

    soup = BeautifulSoup(rq.content, 'lxml')
    builds_dict['Name'] = soup.find('h1', {"class": "build__name"}).text
    comp_table_rows = soup.find('table', {"class": "partlist partlist--mini"}).find_all('tr')
    extra_price = 0

    # Two rows is one component, one for the name of the comp and other for the features
    row_it = iter(comp_table_rows)
    for name, component in zip(row_it, row_it):
        try:
            name_text = name.find('h4').text
            component_el = list(component.find('td', {'class':'td__name'}).children.replace('n', ''))
            # If there is a price set to float value, else set to None
            if len(component_el) > 1:
                comp_price = component_el[1].text
                # If the prices aren't in dollars return empty dict
                if len(comp_price.split(' ')) > 2 or comp_price[0] != '$':
                    print(comp_price)
                    return {}
                comp_price = float(comp_price.replace('$', ''))
            else:
                comp_price = None

            # If the component are in the selected list for scrape
            if name_text in build_comps:
                comp_name = component_el[0].text
                builds_dict[name_text] = {'Name': comp_name, 'Price': comp_price}
            else:
                extra_price += comp_price if isinstance(comp_price, float) else 0

            
        except Exception as e:
            print(e)
            continue

        total_table_row = soup.find('table', {"class": "block partlist partlist--mini partlist--totals"}).find('td', {"class": "td__price"}).text
        builds_dict['Build Price'] = float(total_table_row.replace('$', '')) - extra_price

    return builds_dict

SyntaxError: invalid syntax (<ipython-input-89-32df92961b1f>, line 18)

In [57]:
def main():
    
    browser = get_driver()
    n_pages = 1
    builds_list = [] 
    for i in range(1, n_pages+1):
        try:
            url = parsed_url(page=i)
            browser.get(url)  
            time.sleep(1.2)
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            builds_links = soup.find_all("a", {"class": "logGroup__target"}, href=True)

            for build in builds_links:
                build_url = parsed_url(build_link=build['href'])    
                builds_list.append(build_scraper(build_url))
  
        except Exception as e:
            print(e)
            continue
        
    return builds_list


In [88]:
if __name__ == '__main__':
    #pprint(main())
    # pprint(build_scraper('https://pcpartpicker.com/b/pZzNnQ'))
    pprint(build_scraper('https://pcpartpicker.com/b/2927YJ'))

object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
object of type 'list_iterator' has no len()
{'Name': 'First PC Build'}
